In [1]:
!git clone https://github.com/ngoxuanphong/ENV.git

Cloning into 'ENV'...
remote: Enumerating objects: 1356, done.
remote: Counting objects: 100% (582/582), done.
remote: Compressing objects: 100% (354/354), done.
remote: Total 1356 (delta 239), reused 485 (delta 166), pack-reused 774
Receiving objects: 100% (1356/1356), 67.18 MiB | 18.58 MiB/s, done.
Resolving deltas: 100% (549/549), done.
Updating files: 100% (211/211), done.


In [2]:
%cd ENV

/content/ENV


In [3]:

# Check hệ thống

# import Base.MachiKoro.env as env
# from CheckEnv import check_env
# print(check_env(env))

import numpy as np
import random as rd
from numba import njit, jit
import sys, os
from setup import SHORT_PATH
import importlib.util

import warnings 
warnings.filterwarnings('ignore')
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning,NumbaExperimentalFeatureWarning, NumbaWarning
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaExperimentalFeatureWarning)
warnings.simplefilter('ignore', category=NumbaWarning)


game_name = 'MachiKoro'

In [4]:

def add_game_to_syspath(game_name):
    if len(sys.argv) >= 2:
        sys.argv = [sys.argv[0]]
    sys.argv.append(game_name)

def setup_game(game_name):
    spec = importlib.util.spec_from_file_location('env', f"{SHORT_PATH}Base/{game_name}/env.py")
    module = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = module 
    spec.loader.exec_module(module)
    return module

add_game_to_syspath(game_name)
env = setup_game(game_name)

getActionSize = env.getActionSize
getStateSize = env.getStateSize
getAgentSize = env.getAgentSize

getValidActions = env.getValidActions
getReward = env.getReward
numba_main_2 = env.numba_main_2


In [5]:
        # 1. thẻ lúa mì:                 1 : 1 coin từ bank (blue): 1 coin
        # 2. thẻ nông trại               2 : 1 coin từ bank (blue): 1 coin
        # 3. thẻ tiệm bánh               2-3: 1 coin từ bank (green): 1 coin
        # 4. thẻ quán cà phê             3: 1 coin từ người xúc xắc (red): 2 coins
        # 5. thẻ cửa hàng tiện lợi       4: 3 coins từ bank (green): 2 coins
        # 6. thẻ rừng                    5: 1 coin từ bank (blue): 3 coins
        # 7. thẻ nhà máy pho mát         7: 3 coins từ bank / (mỗi lá nông trại của bạn) (green): 5 coins
        # 8. thẻ nhà máy nội thất        8: 3 coins từ bank / (mỗi lá rừng / quặng của bạn): 3 coins
        # 9. thẻ mỏ quặng                9: 5 coins từ bank (blue): 6 coins
        # 10. thẻ quán ăn gia đình       9-10: 2 coins từ người đổ xx (red): 3 coins
        # 11. thẻ vườn táo               10: 3 coins từ bank (blue): 3 coins
        # 12. thẻ chợ trái cây           11-12: 3 coins từ bank cho mỗi lá ( ) của bạn (green): 2 coins
        # 13. thẻ sân vận động           6: 2 coins từ tất cả người khác (violet): 6 coins
        # 14. thẻ đài truyền hình        6: 5 coins từ 1 người (violet): 7 coins
        # 15. thẻ trung tâm thương mại   6: trao đổi 1 lá k phải () với 1 người khác (violet): 8 coins
        # 16. thẻ 22đ                    có thể đổ lại xí ngầu
        # 17. thẻ 16đ                    nếu đổ xx giống nhau, được đi thêm 1 lượt
        # 18. thẻ 10đ                    tiệm bánh, cafe, quán ăn gđ đc +1 coin
        # 19.thẻ 4đ                     có thể đổ 2 xx

In [6]:
@njit()
def StupidBot(state, per):
    ValidAction = getValidActions(state)
    ValidAction = np.where(ValidAction ==1)[0]

    returnAction = -1
    if 1 in ValidAction:
        returnAction = 1
    elif 43 in ValidAction and state[39] + state[59] + state[79] >= 1 and state[10] <= 0 and state[18] == 1:
        returnAction = 43
    elif 43 in ValidAction and state[39] + state[59] + state[79] >= 3 and state[10] <= 1 and state[18] == 1 :
        returnAction = 43

    elif 35 in ValidAction:
        returnAction = 35
    elif 34 in ValidAction: 
        returnAction = 34
    elif 36 in ValidAction:
        returnAction = 36
    elif 38 in ValidAction:
        returnAction = 38
    elif 39 in ValidAction:
        returnAction = 39
    elif 37 in ValidAction and state[4] <= 2:
        returnAction = 37

    
    # elif 46 in ValidAction and state[18] == 1 and state[13] < 1 and state[0] <= 8:
    #     returnAction = 46
    # elif 47 in ValidAction and state[18] == 1 and state[14] < 1 and state[0] <= 8:
    #     returnAction = 47

    # elif 40 in ValidAction and state[2] >= 2 and state[7] <= 1:
    #     returnAction = 40 

    elif 51 in ValidAction:
        returnAction = 51
    
    # elif 44 in ValidAction and state[11] <= 0 and state[39] + state[59] + state[79] >= 2:
    #     returnAction = 44
    # elif 42 in ValidAction and state[9] <= 0 and state[39] + state[59] + state[79] >= 2:
    #     returnAction = 42

    elif 53 in ValidAction:
        returnAction = 53


    if np.sum(state[1:20]) >= 10:
        if 51 in ValidAction:
            returnAction = 51
        elif 49 in ValidAction:
            returnAction = 49
        elif 50 in ValidAction and np.sum(state[16:19]) == 2:
            returnAction = 50
        elif 52 in ValidAction and np.sum(state[16:19]) == 3:
            returnAction = 52


    # if 0 in ValidAction and (state[117] == 2 or state[117] == 3):
    #         returnAction = 0
    
    if 0 in ValidAction :
        if (state[117] == 2 or state[117] == 4):
            returnAction = 0
        if (state[117] == 3 and state[3]*(1 + state[18]) - state[24]*(1+ state[38]) - state[44]*(1+ state[58]) -state[64]*(1+ state[78]) > 0):
            returnAction = 0
    
    # if returnAction != 51:
    #     if np.sum(state[1:19]) >= 10 and state[18] == 0 and 53 in ValidAction:
    #           returnAction = 53
    


    if returnAction == -1:
      returnAction = ValidAction[np.random.randint(len(ValidAction))]
    #   print('random')
    # print(ValidAction, returnAction)
    # print(state[0:80].reshape(4,20))
    # print(state[80:92])
    # print(state[92:104])
    # print(state[104:116])
    # print(state[116:118])
    # print(state[118:130])
    # print('-----------')

    return returnAction, per

In [7]:
per =[np.zeros((1,1))]

In [9]:
numba_main_2(StupidBot,1000000,per,1)

(464070, [array([[0.]])])